# VADER Sentiment Analysis

In [2]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [3]:
df=pd.read_csv("data/4.12.15-5.11.24_articles.csv")

In [4]:
data = []
for index, row in df.iterrows():
    date_only = row['Date and Time'].split('T')[0]
    data.append({'title': row['Article Title'], 'publishedAt': date_only})

df = pd.DataFrame(data, columns=['title', 'publishedAt'])

In [5]:
df

,title,publishedAt
0,Why Apple Watch Is Just 'Noise' And Apple Inve...,2015-04-12
1,Analyst: Market Is Wrong On Threat To Garmin L...,2015-04-11
2,Weekly Tech Highlights: Apple's TV Service Is ...,2015-04-11
3,"6% Of US Adults Plan To Purchase Apple Watch, ...",2015-04-15
4,"6% of US Adults Plan to Purchase Apple Watch, ...",2015-04-15
...,...,...
16820,Apple's iPad Sales To Get A 'Nice Boost' This ...,2024-05-08
16821,Jim Cramer Advises Investors To Brace For Econ...,2024-05-08
16822,Will The Apple Pencil Pro Work With Your iPad?...,2024-05-08
16823,Apple Inches Closer To Choosing Sam Altman-Led...,2024-05-11


In [6]:
# Initialize VADER
analyzer = SentimentIntensityAnalyzer()

# Function to get sentiments
def get_sentiment(text):
    return analyzer.polarity_scores(text)

# Apply sentiment analysis
df['sentiments'] = df['title'].apply(get_sentiment)
df = pd.concat([df.drop(['sentiments'], axis=1), df['sentiments'].apply(pd.Series)], axis=1)

In [7]:
df

,title,publishedAt,neg,neu,pos,compound
0,Why Apple Watch Is Just 'Noise' And Apple Inve...,2015-04-12,0.000,1.000,0.000,0.0000
1,Analyst: Market Is Wrong On Threat To Garmin L...,2015-04-11,0.394,0.606,0.000,-0.7579
2,Weekly Tech Highlights: Apple's TV Service Is ...,2015-04-11,0.350,0.650,0.000,-0.7906
3,"6% Of US Adults Plan To Purchase Apple Watch, ...",2015-04-15,0.000,0.882,0.118,0.4215
4,"6% of US Adults Plan to Purchase Apple Watch, ...",2015-04-15,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...
16820,Apple's iPad Sales To Get A 'Nice Boost' This ...,2024-05-08,0.000,0.819,0.181,0.4118
16821,Jim Cramer Advises Investors To Brace For Econ...,2024-05-08,0.000,0.788,0.212,0.6249
16822,Will The Apple Pencil Pro Work With Your iPad?...,2024-05-08,0.000,1.000,0.000,0.0000
16823,Apple Inches Closer To Choosing Sam Altman-Led...,2024-05-11,0.000,1.000,0.000,0.0000


In [8]:
data_aggregated = []

# Group by 'publishedAt' date
grouped = df.groupby('publishedAt')

for date, group in grouped:
    # Initialize sentiment sums
    pos_sum = neu_sum = neg_sum = compound_sum = 0
    titles = []

    # Loop through each article in the group
    for _, article in group.iterrows():
        sentiment = get_sentiment(article['title'])
        pos_sum += sentiment['pos']
        neu_sum += sentiment['neu']
        neg_sum += sentiment['neg']
        compound_sum += sentiment['compound']
        titles.append(article['title'])

    # Calculate average sentiments for the group
    n = len(group)
    avg_pos = pos_sum / n
    avg_neu = neu_sum / n
    avg_neg = neg_sum / n
    avg_compound = compound_sum / n

    data_aggregated.append({
        "date": date,
        "article titles": titles,
        "pos": avg_pos,
        "neu": avg_neu,
        "neg": avg_neg,
        "compound": avg_compound
    })

df_aggregated = pd.DataFrame(data_aggregated, columns=['date', 'article titles', 'pos', 'neu', 'neg', 'compound'])

In [9]:
df_aggregated['number of articles'] = df_aggregated['article titles'].apply(lambda x: len(x))

In [10]:
df_aggregated

,date,article titles,pos,neu,neg,compound,number of articles
0,2015-04-11,[Analyst: Market Is Wrong On Threat To Garmin ...,0.000000,0.628000,0.372000,-0.774250,2
1,2015-04-12,[Why Apple Watch Is Just 'Noise' And Apple Inv...,0.000000,1.000000,0.000000,0.000000,1
2,2015-04-14,"[Will The $17,000 Apple Watch Inspire A $20,00...",0.035111,0.945111,0.019778,0.021122,9
3,2015-04-15,"[6% Of US Adults Plan To Purchase Apple Watch,...",0.103286,0.896714,0.000000,0.204386,7
4,2015-04-17,[Apple Offers Workshops For Users To Get Acqua...,0.023700,0.896900,0.079400,-0.057360,10
...,...,...,...,...,...,...,...
2010,2024-05-05,"[Apple's Q2 Earnings, iPhone Sales Slump, And ...",0.000000,1.000000,0.000000,0.000000,1
2011,2024-05-06,"[Thanks To Apple, This Analyst Is Optimistic O...",0.114687,0.863813,0.021437,0.227650,16
2012,2024-05-08,[Apple Reverses Sales Decline in China with Im...,0.114533,0.852533,0.032933,0.225380,15
2013,2024-05-09,"[""Apple Apologizes For iPad 'Crush' Ad That 'M...",0.077000,0.876067,0.046800,0.079860,15


In [11]:
df_aggregated.to_csv('data/4.12.15-5.11.24_VADER_sentiments.csv', encoding='utf-8', index=False)